In [15]:
import os
import json
import pandas as pd
import traceback

In [16]:
from langchain.chat_models import ChatOpenAI

In [17]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [18]:
KEY=os.getenv("OPENAI_API_KEY")

In [20]:
%pip install -U langchain-openai --quiet

Note: you may need to restart the kernel to use updated packages.


In [19]:
from langchain_openai import ChatOpenAI

In [14]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)